In [34]:
import pandas as pd

In [35]:
name = 'RELIANCE'

In [36]:
def files(name: str, no: int):
    df = pd.read_parquet(f'order_{no}.parquet')
    df = df[df['Symbol'] == name]
    return len(df) > 0

def files_seq(name: str):
    i = 1
    res = []
    while True:
        try:
            if files(name, i):
                res.append(i)
            i += 1
        except FileNotFoundError:
            break

        if i > 5:
            break
    return res


print(files_seq(name))

[3, 4]


In [37]:
df1 = pd.read_parquet('order_3.parquet')
df2 = pd.read_parquet('order_4.parquet')

In [38]:
df = pd.concat([df1, df2])

In [39]:
print(len(df) == len(df1) + len(df2))

True


In [40]:
df = df[df['Symbol'] == name]

In [41]:
df.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
16358228,79631194602407,B,866.0,0,Limit,RELIANCE,1,1200000000001802,N,1
16358230,79631194602436,S,1055.0,0,Limit,RELIANCE,1,1200000000001804,N,1
16358250,79631194602723,S,960.0,0,Limit,RELIANCE,24,1200000000001826,N,1
16358258,79631194602855,B,950.0,0,Limit,RELIANCE,1,1200000000001835,N,1
16358259,79631194602868,B,950.0,0,Limit,RELIANCE,1,1200000000001836,N,1


In [42]:
can = pd.read_parquet('cancelled.parquet')

In [43]:
can = can.rename(columns = {'TimeStamp' : 'Timestamp'})

In [44]:
can.head()

,Timestamp,Order Number,IOC Flag,Activity Type
0,79631194995693,1000000000007281,N,3
1,79631195013045,1000000000005540,N,3
2,79631195115036,1000000000006571,N,3
3,79631195116810,1000000000007282,N,3
4,79631195125870,1000000000007860,N,3


In [45]:
can_filtered = can[can['Order Number'].isin(df['Order Number'])]

In [46]:
can_filtered.head()


,Timestamp,Order Number,IOC Flag,Activity Type
6592762,79631195233428,1200000000006171,N,3
6592763,79631195236162,1200000000001836,N,3
6592765,79631195278581,1200000000007951,N,3
6592766,79631195281144,1200000000008332,N,3
6592767,79631195315173,1200000000001835,N,3


In [47]:
len(can_filtered)

85710

In [48]:
data = pd.concat([df, can_filtered], ignore_index=True)


In [49]:
print(len(data) == len(df) + len(can_filtered))

True


In [50]:
data.head(100000000)

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
0,79631194602407,B,866.0,0.0,Limit,RELIANCE,1.0,1200000000001802,N,1
1,79631194602436,S,1055.0,0.0,Limit,RELIANCE,1.0,1200000000001804,N,1
2,79631194602723,S,960.0,0.0,Limit,RELIANCE,24.0,1200000000001826,N,1
3,79631194602855,B,950.0,0.0,Limit,RELIANCE,1.0,1200000000001835,N,1
4,79631194602868,B,950.0,0.0,Limit,RELIANCE,1.0,1200000000001836,N,1
...,...,...,...,...,...,...,...,...,...,...
1495403,79632724984054,NaN,NaN,NaN,NaN,NaN,NaN,1200000005328322,N,3
1495404,79632726947752,NaN,NaN,NaN,NaN,NaN,NaN,1200000005336801,N,3
1495405,79632724986868,NaN,NaN,NaN,NaN,NaN,NaN,1200000005341655,N,3
1495406,79632725516984,NaN,NaN,NaN,NaN,NaN,NaN,1200000005262718,N,3


In [51]:
data.to_parquet(f'{name}_order.parquet')

In [ ]:
df1 = pd.read_parquet('order_2.parquet')

In [26]:
df1 = df1[df1['Order Number'] == '1100000001307852']
df1.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
15720100,79631425757623,S,1348.45,0,Limit,INFY,50,1100000001307852,N,1


In [27]:
can = pd.read_parquet('cancelled.parquet')

In [28]:
can = can[can['Order Number'] == '1100000001307852']
can.head()

,TimeStamp,Order Number,IOC Flag,Activity Type
